In [1]:
!pip install llama-index
!pip install slack_sdk
!pip install qdrant_client


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

from dotenv import find_dotenv, load_dotenv
from llama_index import GPTListIndex, SlackReader

import logging
import structlog
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()
load_dotenv(dotenv_path=find_dotenv(filename=".env.local"), override=True)

/Users/alfie/Documents/code/hash/apps/hash-agents/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("urllib3.util.retry").setLevel(logging.CRITICAL)
logging.getLogger("openai").setLevel(logging.CRITICAL)
logging.getLogger("httpcore").setLevel(logging.CRITICAL)

In [5]:
import json
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

embedding_model = "text-embedding-ada-002"

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(10))
def create_embedding_with_backoff(**kwargs):
    return openai.Embedding.create(**kwargs)

def create_embedding(message, num: int | None, total: int | None):
    logger.debug("Creating embedding", num=num, total=total)
    embedding = create_embedding_with_backoff(
        input=json.dumps(message),
        model=embedding_model,
    )["data"][0]["embedding"]
    return {
        "embedding": embedding,
        "message": message,
    }

In [6]:
from queue import Queue
from collections import namedtuple
from concurrent.futures import Future, ThreadPoolExecutor

from notion_client import Client
from notion_client.helpers import collect_paginated_api

NotionBlockResult = namedtuple("NotionBlockResult", ["content", "block_ids", "sub_pages"])

def traverse_notion_block(
    notion_client: Client,
    task_queue: Queue,
    block_id,
) -> NotionBlockResult:
    print(f"Getting children blocks of {block_id}")
    
    paginated_contents = collect_paginated_api(
        notion_client.blocks.children.list,
        block_id=block_id,
    )

    block_ids = []
    sub_pages = []

    for page in paginated_contents:
        result_obj = page[page["type"]]

        cur_result_text_arr = []
        if "rich_text" in result_obj:
            for rich_text in result_obj["rich_text"]:
                # skip if doesn't have text object
                if "text" in rich_text:
                    text = rich_text["text"]["content"]
                    cur_result_text_arr.append(text)
                elif "plain_text" in rich_text:
                    text = rich_text["plain_text"]
                    cur_result_text_arr.append(text)
        elif "title" in result_obj:
            text = result_obj["title"]
            cur_result_text_arr.append(text)

        if page["has_children"]:
            task_queue.put(page["id"])
            if page["type"] == "child_page":
                sub_pages.append(page["id"])
            else:
                block_ids.append(page["id"])

    return NotionBlockResult(content=" ".join(cur_result_text_arr), block_ids, sub_pages)

SyntaxError: positional argument follows keyword argument (1168879067.py, line 49)

In [ ]:
%%time

import concurrent.futures
import queue
import os
import threading
from notion_client import Client

integration_token = os.getenv("NOTION_API_KEY")
page_ids = ["964db53ace9940519613dce0485820fd"]

notion_client = Client(auth=integration_token) 

executor = concurrent.futures.ThreadPoolExecutor()
task_queue = queue.Queue()
results = {}

def process_queue():
    while True:
        block_id = task_queue.get()
        results[block_id] = executor.submit(traverse_notion_block, notion_client, task_queue, block_id)
        task_queue.task_done()

queue_thread = threading.Thread(target=process_queue)
queue_thread.start()

for page_id in page_ids:
    task_queue.put(page_id)

task_queue.join()

In [ ]:
results

In [11]:
pages_to_explore = [*page_ids]
explored_pages = set()
pages_to_contents = {}


def collapse_text(root_block_id, depth):
    block_result = results[root_block_id].result()
    pages_to_explore.extend(block_result.sub_pages)

    block_content = block_result.content
    nested_content = "\n".join(
        [collapse_text(block_id, depth + 1) for block_id in block_result.block_ids]
    )
    return (depth * "\t") + block_content + nested_content


while len(pages_to_explore) > 0:
    page_id = pages_to_explore.pop(0)
    if page_id in explored_pages:
        continue
    explored_pages.add(page_id)

    print(f"Exploring page {page_id}")
    pages_to_contents[page_id] = collapse_text(page_id, 0)

In [ ]:
def create_payload(document):
    return {
        "text": document.text if document.text else None,
        "doc_id": document.doc_id if document.doc_id else None,
        "extra_info": document.extra_info if document.extra_info else None
    }

In [ ]:
embedding = create_embedding(message=create_payload(documents[0]), num=0, total=1)

In [ ]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor() as executor:
    embeddings = list(
        executor.map(
            lambda idx_item: create_embedding(
                message=create_payload(idx_item[1]),
                num=idx_item[0],
                total=len(documents),
            ),
            enumerate(documents),
        ),
    )

In [ ]:
from llama_index import LLMPredictor, ServiceContext, PromptHelper
from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
prompt_helper = PromptHelper.from_llm_predictor(llm_predictor=llm_predictor)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    chunk_size_limit=1024,
    prompt_helper=prompt_helper,
)

In [ ]:
from llama_index import GPTQdrantIndex
import qdrant_client

client = qdrant_client.QdrantClient(
    url="http://localhost:6333"
)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("My docker-compose isn't working, how can I fix it?")
response

In [ ]:
response.response